In [34]:
import os
import numpy as np
from matplotlib import pyplot as plt
import pickle

In [45]:
db = "14"
window_size = "10"
lag = "2"
features = "polar"
save_dir = f"db{db}/window-size-{num_windows}/lag{lag}/{features}-features/"

In [51]:
size = 75

In [35]:
def savePickle(data, filename):
    with open(os.getcwd() + "/train-val-test/" + filename, "wb") as f: np.save(f, data)

In [36]:
class User:
    def __init__(self, json_doc):
        self.subject_id = json_doc["subject_id"]
        self.name = json_doc["name"]
        self.yob = json_doc["yob"]
        self.height = json_doc["height"]
        self.weight = json_doc["weight"]
        self.is_female = json_doc["is_female"]
        self.medical_conditions = json_doc["medical_conditions"]

In [37]:
if os.path.exists(os.getcwd() + "/pickles/db14/users.p"):
    users = pickle.load(open(os.getcwd() + "/pickles/db14/users.p", "rb"))

In [52]:
class Recording:
    def __init__(self, meta):
        self._id = meta._id
        self.subject_id = meta.subject_id
        self.phone_placement = meta.phone_placement
        self.recording_duration = meta.recording_duration
        self.chunk_ids = meta.chunk_ids
        self.labels = []
        self.p_ecg = []
        self.p_hr = []
        self.p_contact = []
        self.p_acc_x = []
        self.p_acc_y = []
        self.p_acc_z = []
        self.acc_x = []
        self.acc_y = []
        self.acc_z = []
        self.gyr_x = []
        self.gyr_y = []
        self.gyr_z = []
        self.gra_x = []
        self.gra_y = []
        self.gra_z = []
        self.mag_x = []
        self.mag_y = []
        self.mag_z = []
        self.att_roll = []
        self.att_pitch = []
        self.att_yaw = []
        self.delta_heading = []
        self.height = users[self.subject_id].height
        self.weight = users[self.subject_id].weight
        
        print(self._id)
        r_chunks = rec_chunks[self._id]
        for c in r_chunks:
            print(c._id)
        print()
        ordered_chunk_ids = {}
        
        for chunk in r_chunks:
            ordered_chunk_ids[chunk.chunk_index] = chunk
            
        for i in range(1,len(r_chunks)-1):
            chunk = chunks[ordered_chunk_ids[i]._id]
            
            assert(i == chunk.chunk_index)
            
            self.labels += chunk.labels
            self.p_ecg += chunk.p_ecg
            self.p_hr += chunk.p_hr
            self.p_contact += chunk.p_contact
            self.p_acc_x += chunk.p_acc_x
            self.p_acc_y += chunk.p_acc_y
            self.p_acc_z += chunk.p_acc_z
            self.acc_x += chunk.acc_x
            self.acc_y += chunk.acc_y
            self.acc_z += chunk.acc_z
            self.gyr_x += chunk.gyr_x
            self.gyr_y += chunk.gyr_y
            self.gyr_z += chunk.gyr_z
            self.gra_x += chunk.gra_x
            self.gra_y += chunk.gra_y
            self.gra_z += chunk.gra_z
            self.mag_x += chunk.mag_x
            self.mag_y += chunk.mag_y
            self.mag_z += chunk.mag_z
            self.att_roll += chunk.att_roll
            self.att_pitch += chunk.att_pitch
            self.att_yaw += chunk.att_yaw
            self.delta_heading += chunk.delta_heading
            
    def getIntervals(self, lag):
        output = np.zeros((len(self.labels), size+1))
        returnIdx = -1
        
        for i in range(len(self.labels)):
            data = IntervalData(self, i).flatten()
            if data.shape[0] == size+1:
                output[i,:] = data
            else:
                returnIdx = i
                break
        
        if returnIdx != -1:
            new_output = output[:returnIdx,:]
            
            return new_output[:-lag,:-1], new_output[lag:,-1]
        
        else:
            return output[:-lag,:-1], output[lag:,-1]

In [56]:
class IntervalData:
    def __init__(self, rec, idx):
        self.label = [rec.labels[idx]]
        self.p_ecg = rec.p_ecg[idx*13 : idx*13 + 13]
        self.p_acc_x = rec.p_acc_x[idx*20 : idx*20 + 20]
        self.p_acc_y = rec.p_acc_y[idx*20 : idx*20 + 20]
        self.p_acc_z = rec.p_acc_z[idx*20 : idx*20 + 20]
        self.height = rec.height
        self.weight = rec.weight
        if idx < len(rec.acc_x):
            self.acc_x = [rec.acc_x[idx]]
            self.acc_y = [rec.acc_y[idx]]
            self.acc_z = [rec.acc_z[idx]]
            self.gyr_x = [rec.gyr_x[idx]]
            self.gyr_y = [rec.gyr_y[idx]]
            self.gyr_z = [rec.gyr_z[idx]]
            self.gra_x = [rec.gra_x[idx]]
            self.gra_y = [rec.gra_y[idx]]
            self.gra_z = [rec.gra_z[idx]]
            self.mag_x = [rec.mag_x[idx]]
            self.mag_y = [rec.mag_y[idx]]
            self.mag_z = [rec.mag_z[idx]]
            self.att_roll = [rec.att_roll[idx]]
            self.att_pitch = [rec.att_pitch[idx]]
            self.att_yaw = [rec.att_yaw[idx]]
            self.delta_heading = [rec.delta_heading[idx]]
        else:
            self.acc_x = []
            self.acc_y = []
            self.acc_z = []
            self.gyr_x = []
            self.gyr_y = []
            self.gyr_z = []
            self.gra_x = []
            self.gra_y = []
            self.gra_z = []
            self.mag_x = []
            self.mag_y = []
            self.mag_z = []
            self.att_roll = []
            self.att_pitch = []
            self.att_yaw = []
            self.delta_heading = []
        
    def flatten(self):
        #size = 17*1 + 13*1 + 20*3
        output = np.zeros((size+1))
        
        if len(self.p_ecg) == 13:
            output[0:13] = np.array(self.p_ecg)
        else:
            return np.zeros((0))
        if len(self.p_acc_x) == 20:
            output[13:33] = np.array(self.p_acc_x)
            output[33:53] = np.array(self.p_acc_y)
            output[53:73] = np.array(self.p_acc_z)
        else:
            return np.zeros((0))
        if len(self.label) == 1:
            output[73] = self.height
            output[74] = self.weight
            output[75] = self.label[0]
        else:
            return np.zeros((0))
        
        return output
        

# Read Pickle Data

In [54]:
ddir = os.getcwd() + f"/pickles/db{db}"
files = os.listdir(ddir)
recordings = []

for f in files:
    if not "users.p" in f:
        reader = open(ddir + "/" + f, "rb")
        rec = pickle.load(reader)
        print(rec._id)
        recordings.append(rec)

AEA56528-8D55-49B1-B0DC-A5CCB861850E
894DB681-F91D-4568-A027-EA97D234F274
00160B9F-7725-41D0-BF42-40679D1FF8C0
1909DCBF-C4B6-4535-9996-5519C03C1C7A
E4ED61E8-E9DD-41F8-9E15-F9054C0DAE84
A43647D0-72C1-4547-89B5-AE7198EBD909
D47BF108-145A-48CE-B05C-A8D57F539CD3
08F36872-722B-4140-8CB8-ACA17C34358E
B6B2C985-1C0A-4FC7-9E8C-E2F05887E57B
3B1567A4-5750-4AAA-B9D5-71C8C6674B24
4D5F47D2-E7D2-4D15-8081-F7BAE8BBCC94
DC15019A-2F4D-4583-B886-1C9ABC01E40A
24E5BE1A-F04C-40B3-8ADB-AF8922DADD7A
6C6F17B7-7225-4124-95D1-B214D7BCC00C
47AE8975-9C88-4B3F-88AB-4899C7FF3DBF
7332DDD8-486C-4C33-99CE-00EB01669F9A
716D3350-7A76-4F77-BFD8-D0D563B8D3E0
001C6F5A-901D-4CD8-8EF9-B8E0AF0548CB
1B4C354E-3CAE-4986-B325-9486F3E664BC
987E552F-86BA-43D7-AB46-1864A3FD01E0
64A2056B-BC1C-4E6F-A641-43DF216EA153
40181EF2-1A30-4C0B-81D2-9773E158F5C0
C7782005-0FF7-4F34-B10D-F8BA655BA7B6
666F15F0-2E75-43BB-8DAD-8CD223375F07
36D1280E-8412-4747-959D-63078BE2EAF3


# Parse Data Into Intervals

In [57]:
intvls = []
for r in recordings:
    print(r._id)
    intvls.append(r.getIntervals(int(lag)))

AEA56528-8D55-49B1-B0DC-A5CCB861850E
894DB681-F91D-4568-A027-EA97D234F274
00160B9F-7725-41D0-BF42-40679D1FF8C0
1909DCBF-C4B6-4535-9996-5519C03C1C7A
E4ED61E8-E9DD-41F8-9E15-F9054C0DAE84
A43647D0-72C1-4547-89B5-AE7198EBD909
D47BF108-145A-48CE-B05C-A8D57F539CD3
08F36872-722B-4140-8CB8-ACA17C34358E
B6B2C985-1C0A-4FC7-9E8C-E2F05887E57B
3B1567A4-5750-4AAA-B9D5-71C8C6674B24
4D5F47D2-E7D2-4D15-8081-F7BAE8BBCC94
DC15019A-2F4D-4583-B886-1C9ABC01E40A
24E5BE1A-F04C-40B3-8ADB-AF8922DADD7A
6C6F17B7-7225-4124-95D1-B214D7BCC00C
47AE8975-9C88-4B3F-88AB-4899C7FF3DBF
7332DDD8-486C-4C33-99CE-00EB01669F9A
716D3350-7A76-4F77-BFD8-D0D563B8D3E0
001C6F5A-901D-4CD8-8EF9-B8E0AF0548CB
1B4C354E-3CAE-4986-B325-9486F3E664BC
987E552F-86BA-43D7-AB46-1864A3FD01E0
64A2056B-BC1C-4E6F-A641-43DF216EA153
40181EF2-1A30-4C0B-81D2-9773E158F5C0
C7782005-0FF7-4F34-B10D-F8BA655BA7B6
666F15F0-2E75-43BB-8DAD-8CD223375F07
36D1280E-8412-4747-959D-63078BE2EAF3


# Shift and scale X data

In [75]:
x_mu = np.zeros((size))
rows = 0

for i, xy in enumerate(intvls):
    x_mu += np.sum(xy[0], axis=0)
    rows += xy[0].shape[0]
    
x_mu = x_mu/rows

In [17]:
x_mu = np.zeros((size))
x_std = np.zeros((size))
rows = 0

for i, xy in enumerate(intvls):
    x_mu += np.sum(xy[0], axis=0)
    x_std += np.std(xy[0], axis=0)
    rows += xy[0].shape[0]
    
x_mu = x_mu/rows
x_std = x_std/rows

In [58]:
x_min = np.zeros((size)) + 10000
x_max = np.zeros((size))

for i, xy in enumerate(intvls):
    print(xy[0])
    xmin = np.min(xy[0], axis=0)
    for j, x in enumerate(xmin):
        if x < x_min[j]:
            x_min[j] = x
    
    xmax = np.max(xy[0], axis=0)
    for k, x in enumerate(xmax):
        if x > x_max[k]:
            x_max[k] = x

savePickle(x_min, f"{save_dir}x_min.npy")
savePickle(x_max, f"{save_dir}x_max.npy")

[[-171. -113. -211. ...  149.  180.   77.]
 [  16.   33.   31. ...  145.  180.   77.]
 [ -84.  -89.  -86. ...  246.  180.   77.]
 ...
 [ 132.  144.  166. ...   88.  180.   77.]
 [ -33.  -26.   19. ...   62.  180.   77.]
 [ -19. -144. -431. ...   30.  180.   77.]]
[[1948.  912. -445. ...  -66.  170.   72.]
 [  24.   33.   36. ...  137.  170.   72.]
 [ 192.  199.  175. ...  183.  170.   72.]
 ...
 [-178. -120. -137. ... -602.  170.   72.]
 [ 166.  151.  132. ... -584.  170.   72.]
 [-103. -110. -105. ... -327.  170.   72.]]
[[ 199.  192.  166. ...  210.  195.   94.]
 [ 103.  130.  115. ...  175.  195.   94.]
 [ 183.  816. 1546. ...  158.  195.   94.]
 ...
 [  48.  -26. -118. ... -407.  195.   94.]
 [-226. -219. -195. ... -473.  195.   94.]
 [-219. -219. -224. ... -240.  195.   94.]]
[[    4.    14.    43. ...    12.   180.    77.]
 [    4.   -55.  -120. ...   133.   180.    77.]
 [ -125.  -101.   -77. ...   151.   180.    77.]
 ...
 [  -52.   -74.  -134. ...  -355.   180.    77.]
 [ -127

# Sliding Windows

In [59]:
import more_itertools as mit

def window(seq, n=10):
    return np.array([list(mit.flatten(s)) for s in mit.windowed(seq, n)])

def window_1d(seq, n=10):
    return np.lib.stride_tricks.sliding_window_view(seq, window_shape = n)

In [60]:
ys = []
ys_w = []
xs = []

for x,y in intvls:
    print(x.shape)
    print(y.shape)
    #xs.append(window((x - x_mu)/x_std, num_windows))
    xs.append(window((x - x_min)/(x_max - x_min), num_windows))
    ys_w.append(window_1d(y, num_windows))
    ys.append(ys_w[-1][:,0])
    print(xs[-1].shape)
    print(ys_w[-1].shape)
    print(ys[-1].shape)
    print()

(697, 75)
(697,)
(688, 750)
(688, 10)
(688,)

(1557, 75)
(1557,)
(1548, 750)
(1548, 10)
(1548,)

(1146, 75)
(1146,)
(1137, 750)
(1137, 10)
(1137,)

(348, 75)
(348,)
(339, 750)
(339, 10)
(339,)

(1797, 75)
(1797,)
(1788, 750)
(1788, 10)
(1788,)

(896, 75)
(896,)
(887, 750)
(887, 10)
(887,)

(370, 75)
(370,)
(361, 750)
(361, 10)
(361,)

(348, 75)
(348,)
(339, 750)
(339, 10)
(339,)

(1297, 75)
(1297,)
(1288, 750)
(1288, 10)
(1288,)

(1348, 75)
(1348,)
(1339, 750)
(1339, 10)
(1339,)

(1897, 75)
(1897,)
(1888, 750)
(1888, 10)
(1888,)

(1346, 75)
(1346,)
(1337, 750)
(1337, 10)
(1337,)

(972, 75)
(972,)
(963, 750)
(963, 10)
(963,)

(1197, 75)
(1197,)
(1188, 750)
(1188, 10)
(1188,)

(997, 75)
(997,)
(988, 750)
(988, 10)
(988,)

(1847, 75)
(1847,)
(1838, 750)
(1838, 10)
(1838,)

(1346, 75)
(1346,)
(1337, 750)
(1337, 10)
(1337,)

(1497, 75)
(1497,)
(1488, 750)
(1488, 10)
(1488,)

(1622, 75)
(1622,)
(1613, 750)
(1613, 10)
(1613,)

(1596, 75)
(1596,)
(1587, 750)
(1587, 10)
(1587,)

(1397, 75)
(139

## Merge windows

In [61]:
num_rows = np.sum([x.shape[0] for x in xs])
print(num_rows)

28954


In [63]:
xx = np.zeros((num_rows, num_windows*size))
yy = np.zeros((num_rows))

offset = 0
for i in range(len(xs)):
    assert(xs[i].shape[0] == ys[i].shape[0])
    
    xx[offset : offset+xs[i].shape[0], :] = xs[i]
    yy[offset : offset+xs[i].shape[0]] = ys[i]
    offset += xs[i].shape[0]

In [64]:
yy_re = np.zeros((num_rows))

offset = 0
for i in range(len(xs)):
    assert(xs[i].shape[0] == ys_w[i].shape[0])
    
    for j in range(xs[i].shape[0]):
        yy_re[offset+j] = np.sum([(2*ys_w[i][j,k])**(num_windows-k) for k in range(num_windows)])
        
    offset += xs[i].shape[0]
    
yy_re = yy_re - np.mean(yy_re)
yy_re = yy_re / np.std(yy_re)

In [65]:
yy_rs = np.zeros((num_rows))

offset = 0
for i in range(len(xs)):
    assert(xs[i].shape[0] == ys_w[i].shape[0])
    yy_rs[offset : offset+xs[i].shape[0]] = np.sum(ys_w[i], axis=1)
    offset += xs[i].shape[0]
    
yy_rs = yy_rs - np.mean(yy_rs)
yy_rs = yy_rs / np.std(yy_rs)

In [66]:
from sklearn.model_selection import train_test_split

X_tmp, X_test, y_tmp, y_test = train_test_split(xx, yy, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.25, train_size=0.75, random_state=42, shuffle=True)

In [67]:
savePickle(X_train, f"{save_dir}X_train.npy")
savePickle(X_test, f"{save_dir}X_test.npy")
savePickle(X_val, f"{save_dir}X_val.npy")
savePickle(y_train, f"{save_dir}y_train.npy")
savePickle(y_test, f"{save_dir}y_test.npy")
savePickle(y_val, f"{save_dir}y_val.npy")

In [68]:
Xre_tmp, Xre_test, yre_tmp, yre_test = train_test_split(xx, yy_re, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)
Xre_train, Xre_val, yre_train, yre_val = train_test_split(Xre_tmp, yre_tmp, test_size=0.25, train_size=0.75, random_state=42, shuffle=True)

In [69]:
savePickle(Xre_train, f"{save_dir}Xre_train.npy")
savePickle(Xre_test, f"{save_dir}Xre_test.npy")
savePickle(Xre_val, f"{save_dir}Xre_val.npy")
savePickle(yre_train, f"{save_dir}yre_train.npy")
savePickle(yre_test, f"{save_dir}yre_test.npy")
savePickle(yre_val, f"{save_dir}yre_val.npy")

In [70]:
Xrs_tmp, Xrs_test, yrs_tmp, yrs_test = train_test_split(xx, yy_rs, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)
Xrs_train, Xrs_val, yrs_train, yrs_val = train_test_split(Xrs_tmp, yrs_tmp, test_size=0.25, train_size=0.75, random_state=42, shuffle=True)

In [71]:
savePickle(Xrs_train, f"{save_dir}Xrs_train.npy")
savePickle(Xrs_test, f"{save_dir}Xrs_test.npy")
savePickle(Xrs_val, f"{save_dir}Xrs_val.npy")
savePickle(yrs_train, f"{save_dir}yrs_train.npy")
savePickle(yrs_test, f"{save_dir}yrs_test.npy")
savePickle(yrs_val, f"{save_dir}yrs_val.npy")

In [32]:
xx.shape

(28952, 910)